# 1. The data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#path = '/home/ahardy/ENSchallengeData/' 

X_train = pd.read_csv('./X_train.csv', index_col=0, sep=',')
X_train.columns.name = 'date'

Y_train = pd.read_csv('./Y_train.csv', index_col=0, sep=',')
Y_train.columns.name = 'date'

In [4]:
X_train.shape

(50, 754)

In [3]:
X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

C:\Users\user\AppData\Local\Temp\ipykernel_4408\1489607172.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


# 2.Some functions we will need taken from the QRT notebook

In [20]:
# to check the orthonormality constraints as in the metric:
    
def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - np.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool

In [21]:
def randomA(D=250, F=10):  
    
    M = np.random.randn(D,F)
    randomStiefel = np.linalg.qr(M)[0] # Apply Gram-Schmidt algorithm to the columns of M
    
    return randomStiefel

In [19]:
# Given a Stiefel matrix A returns the optimal \beta

def fitBeta(A):
    
    predictors = X_train_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
    targets = Y_train.T.stack()
    beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets
    
    return beta.to_numpy()

In [18]:
#Given a Stiefel matrix A returns the metric on the training set. 

def metric_train(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    Ypred = (X_train_reshape @ A @ beta).unstack().T         
    Ytrue = Y_train
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap  

# 3. The roadmap.

  We first reformulate the problem in the following way: given X_train_reshape and Ytrue, our goal is to minimize the function 
 $F: M_{250}^{10} \to \mathbb{R}$ defined by
 
 $F(A) = - \frac{1}{504} \sum_{t=250}^{753} \frac{ <\tilde{S}_t(A),\tilde{R}_t>}{||\tilde{S}_t(A)|| ||\tilde{R}_t||}$ where:
 
  _ $M_{250}^{10}$ is the Stiefel manifold, namely the set of $250 \times 10$ matrices such that $A^TA = I_{10}$.
  
  _ $\tilde{S}_t$ is our prediction for  $\tilde{R}_t$ in the setup of Stiefel matrices. For each fixed $t$, $\tilde{S}_t$ and $\tilde{R}_t$ are elements of $\mathbb{R}^{50}$.
  
   The way the vectors $(\tilde{S}_t)_{t=250, \ldots, 753}$ are constructed given the matrix $A$ is quite simple: 
   
   _ First compute the explicative factors associated to $A$, that is 
 
  $F_{t,l} = \sum_{k=1}^{10} A_{k,l} R_{t+1-l}$ for $l=1,2, \ldots, 10$.
  
   _ Then optimize the vector $\beta \in \mathbb{R}^{10}$ with respect to the observation using linear regression, that is find the optimal $\beta$'s for the relation 
    
  $S_{t+1} = \sum_{l=1}^{10} \beta_l F_{t,l}$ to hold as accurately as possible. This provides a vector $\beta$, which we use to construct our prediction
  
  $\tilde{S}_{t+1} = \sum_{l=1}^{10} \beta_l F_{t,l}$. 
  
  
Note that given $A$, the vector $\beta$ is actually uniquely determined; in other words, we can write $\beta = \beta(A)$. Actually, with the notations of the notebook (see the definition of fitBeta), if we define 
 predictors = X_train_reshape @ A and targets = Y_train.T.stack(), we have the following:
 
  beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets.
  
   This writes mathematically 
  
 $ \beta =  \beta(A) = [(X_{tr} A)^T(X_{tr} A)]^{-1} (X_{tr} A)^T Y_{pred} $

 where $X_{tr}$ is the vectors of delayed stock prices and $Y_{pred}$ are the associated predictions. In a similar way, for $t \in [250,753]$, we can write 

 $\frac{ <\tilde{S}_t(A),\tilde{R}_t>}{||\tilde{S}_t(A)|| ||\tilde{R}_t||} = f(g(A))$ 

 where $f: \mathbb{R}^{50} \to \mathbb{R}$ is defined by $f(x) = \frac{x.r}{||x||}$ where we chose to write $r = \frac{\tilde{R}_t}{||\tilde{R}_t||}$ and $g : \mathbb{R}^{250 \times 10} \to \mathbb{R}^{50}$ 
 is defined by $g(A) = X_{tr} A \beta(A)$. 

 
  
  

 
  
 
  
     
 We very much would like to implement some sort of gradient descent on the function $F$. Of course, this is not straightforward since making a step in any direction is going to leave the Stiefel manifold, so that we need to ensure that we proceed along the manifold. Fortunately, a 2010 paper provides such an algorithm. For this algorithm to work, the first step is to compute explicitly the gradient of the function $F$ at any given point $A$ of the manifold which we denote by $ G = (\frac{\partial F}{\partial A_{i,j}})_{i,j}$ which is a $250 \times 10$ matrix. 
 
  Getting an exact expression for this gradient is a bit cumbersome, so that in first approximation we assume $\beta$ to be locally constant (and will update its value accordingly for each step of our algorithm). Under this assumption (which might prove quite terrible, I don't know), the computation of the gradient merely reduces to some easy derivatives (no inversion matrices needed, as would be the case for differentiating $\beta$), and we get the expression:
 
 $G = C * \beta.T$ where $C = \sum_{j=250}^{753} C_j$ and $C_j = \frac{1}{||R_{j}||.||\tilde{S}_{j}||}. [R_{j}-\frac{(R_{j}.\tilde{S}_{j})}{||\tilde{S}_{j}||^2} \tilde{S}_{j}] ** R_j^{past}$. We denoted by $R_j^{past}$ the vector $X_{train reshape}.T[:,j]$ which is a 2520 * 1 vector, and $**$ is the component by component scalar product (that is, $a ** (b_1,b_2) = (a.b_1,a.b_2))$).
 
  The algorithm then proceeds as follows:
  
  _ First we define a skew symmetric matrix $W = G A^T - A G^T$ of size $250 \times 250.$
  
  _ Then we proceed along the curve $Y(t) = (I_{250} + t W)^{-1} (I_{250} - t W) A$. The very nice thing is that for any $t$ (let us say small enough), $Y(t)$ is an element of $M_{250}^{10}$.

First we define a useful fonction for the computation of the gradient. Given a and b vectors, returns  $\frac{1}{||a||.||b||}. [a-\frac{(a.b)}{||b||^2} b]$ (note that of course this is NOT symmetric in $a,b$)

In [69]:
def ps(A,B,D=250):
    Y = [A - (A * B).sum()/((B**2).sum())*B].div(np.sqrt((A**2).sum()) * np.sqrt((B**2).sum()))
    return Y 

    

Then we construct the matrix C

In [ ]:
def 

# 5. Computing the gradient of F.

In [7]:
U = np.random.randn(5,5)
U
U_pd = pd.DataFrame(U,columns=range(5))

In [8]:
U_pd

,0,1,2,3,4
0,-0.780047,-0.067963,-0.517825,-0.651537,1.300125
1,1.399684,0.577422,1.737584,-2.588198,-0.930516
2,-1.821823,-1.099561,-1.324913,-0.922661,-0.970990
3,-1.225205,-2.267096,0.828100,-0.715108,-0.882429
4,0.244252,0.579590,-0.091930,-0.145283,-0.295153


In [16]:
X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()

C:\Users\user\AppData\Local\Temp\ipykernel_4408\3997061981.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


In [28]:
U_pd.T.stack()

0  0   -0.780047
   1    1.399684
   2   -1.821823
   3   -1.225205
   4    0.244252
1  0   -0.067963
   1    0.577422
   2   -1.099561
   3   -2.267096
   4    0.579590
2  0   -0.517825
   1    1.737584
   2   -1.324913
   3    0.828100
   4   -0.091930
3  0   -0.651537
   1   -2.588198
   2   -0.922661
   3   -0.715108
   4   -0.145283
4  0    1.300125
   1   -0.930516
   2   -0.970990
   3   -0.882429
   4   -0.295153
dtype: float64

array([[ 0.        ,  0.35706273, -0.0247103 ,  1.23604444,  0.40647254],
       [-0.35706273,  0.        , -4.64260956, -1.11259063, -1.15651935],
       [ 0.0247103 ,  4.64260956,  0.        ,  4.65328498,  0.25450636],
       [-1.23604444,  1.11259063, -4.65328498,  0.        ,  0.99147823],
       [-0.40647254,  1.15651935, -0.25450636, -0.99147823,  0.        ]])

In [13]:
range(-100,100)

range(-100, 100)

In [22]:
A = randomA()
beta = fitBeta(A)
m = metric_train(A, beta)

In [23]:
m

0.009989403184319668

In [24]:
U = np.random.randn(250,250)
W = U - U.T #generating a skew symmetric random matrix at each step

In [35]:
t=90

In [36]:
Y =  np.linalg.inv(np.eye(250) - t/1000*W) @  (np.eye(250) - t/1000*W) @ A
beta = fitBeta(Y)
m = metric_train(Y,beta)

In [37]:
m

0.00998940318431968

In [41]:
I = np.eye(3)

In [42]:
I

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [43]:
import pickle 

In [63]:
import pickle 
with open('mypicklefile', 'wb') as f1:
    pickle.dump(I, f1)

In [53]:
with open('mypicklefile', 'r') as f1:
    f1.read()

In [64]:
with open('mypicklefile', 'rb') as f1:
    XAR = pickle.load(f1)

In [65]:
XAR

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [61]:
I = np.eye(14)

In [62]:
I

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])